In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [ ]:
file_path = "/content/drive/MyDrive/Neuraaal/Assign_5/mail_data.csv"
data = pd.read_csv(file_path)

In [ ]:
# Preprocessing
label_encoder = LabelEncoder()
data['Category'] = label_encoder.fit_transform(data['Category'])

In [ ]:
X = data['Message']
y = data['Category']

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

In [ ]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_train_padded = pad_sequences(X_train_seq, maxlen=100)
X_test_padded = pad_sequences(X_test_seq, maxlen=100)

In [ ]:
# Build RNN model
model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=100),
    LSTM(64, return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train_padded, y_train, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 21s 150ms/step - accuracy: 0.8919 - loss: 0.3219 - val_accuracy: 0.9675 - val_loss: 0.1160
Epoch 2/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 17s 150ms/step - accuracy: 0.9883 - loss: 0.0449 - val_accuracy: 0.9809 - val_loss: 0.0772
Epoch 3/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 16s 145ms/step - accuracy: 0.9949 - loss: 0.0244 - val_accuracy: 0.9787 - val_loss: 0.0903
Epoch 4/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 16s 145ms/step - accuracy: 0.9982 - loss: 0.0134 - val_accuracy: 0.9798 - val_loss: 0.1015
Epoch 5/5
112/112 ━━━━━━━━━━━━━━━━━━━━ 24s 180ms/step - accuracy: 0.9989 - loss: 0.0045 - val_accuracy: 0.9809 - val_loss: 0.0892


In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test)
print(f"Test Accuracy: {accuracy}")

35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.9879 - loss: 0.0597
Test Accuracy: 0.9892376661300659


In [ ]:
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
import numpy as np

# Evaluate the model
loss, accuracy = model.evaluate(X_test_padded, y_test, verbose=0)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

Test Loss: 0.0572853609919548
Test Accuracy: 0.9892376661300659


In [ ]:
# Predict on test data
y_pred_probs = model.predict(X_test_padded)
y_pred = (y_pred_probs > 0.5).astype(int).flatten()

35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step


In [ ]:
# Calculate precision, recall, and F1-score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [ ]:
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Precision: 0.9928057553956835
Recall: 0.9261744966442953
F1 Score: 0.9583333333333334


In [ ]:
# Classification report for a detailed breakdown
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Ham', 'Spam']))


Classification Report:
              precision    recall  f1-score   support

         Ham       0.99      1.00      0.99       966
        Spam       0.99      0.93      0.96       149

    accuracy                           0.99      1115
   macro avg       0.99      0.96      0.98      1115
weighted avg       0.99      0.99      0.99      1115



In [ ]:
# Function to preprocess and predict user input
def predict_email(email_text):
    # Tokenize and pad the input email text
    email_seq = tokenizer.texts_to_sequences([email_text])
    email_padded = pad_sequences(email_seq, maxlen=100)

    # Predict using the trained model
    prediction = model.predict(email_padded)
    return "Spam" if prediction > 0.5 else "Ham"

# Take user input
print("\n=== Email Spam Detection ===")
while True:
    user_input = input("Enter an email to classify (or type 'exit' to quit): ").strip()
    if user_input.lower() == 'exit':
        break
    result = predict_email(user_input)
    print(f"Prediction: {result}\n")



=== Email Spam Detection ===
Enter an email to classify (or type 'exit' to quit): Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Prediction: Ham

Enter an email to classify (or type 'exit' to quit): Upgrade to our premium plan for exclusive access to premium content and features.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Prediction: Spam

Enter an email to classify (or type 'exit' to quit): You're a winner! Click here to claim your exclusive prize.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Prediction: Spam

Enter an email to classify (or type 'exit' to quit): Thank you for your feedback. We're always striving to improve our services.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Prediction: Ham

Enter an email to classify (or type 'exit' to quit): We're sorry for the inconvenience. Our website will be back online shortly.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Prediction: Ham

Enter an email to class